# Generate intraday bars using bars data service - C#

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot `intraday bars` with a **customizable** time range sampling.<br>

It enables to retrieve **on-demand** data by calling a dedicated bars service on a tick by tick data store. 

### Inputs/outputs
Bars sample requires instrument's identifier, date time intervals as per inputs. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve bars from the dedicated hosted service. The queried endpoint in this script are:
* *TickBarsService*: to directly retrieve bars objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run bars sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick bars service*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Bars request parameters

#### 3.1 Instrument selection

In [4]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Bars calculation parameters

In [5]:
// Set the bar duration
var sampling = 5 * 60;

// Set the bar calculation field
var field = BarPrice.Trade; 

#### 3.3 Time period delimitation

In [6]:
// Create time intervals
var yesterday = DateTime.Today.AddDays(-1);
var start = yesterday.AddDays(-50);

// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = yesterday.Year, Month = yesterday.Month, Day = yesterday.Day }
};

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
 StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
 EndTime = new TimeOfDay { Hours = 20, Minutes = 00, Seconds = 00 } 
};

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [7]:
// Generate constraints based on the previous space selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

In [8]:
// Generate intraday bars request
var request = new TickBarsRequest  
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,  
    Sampling = new Duration {Seconds = sampling},
    Field = field
};

In [9]:
// Instantiate the intraday bars service
var service = new TickBarsService.TickBarsServiceClient(channel);

In [10]:
// Process intraday bars request
var bars = new List<TickBarsResponse>();

var call = service.TickBars(request, headers);
await foreach (var bar in call.ResponseStream.ReadAllAsync())
{
    bars.Add(bar);
}

In [11]:
// Calculate the total ticks inside all the bars we received
var ticks = bars.Sum(bar => bar.Count);

// Display
Console.WriteLine($"Fetched {bars.Count:N0} bars from (Total trades={ticks:N0})");

Fetched 432 bars from (Total trades=236,735)


In [12]:
// Display the data
//--> Note : the time stamp is displayed by default (using the unix format)
//--> We can use other displayers for different output formats 
//--> For more details about the time stamp: https://developers.google.com/protocol-buffers/docs/reference/google.protobuf#google.protobuf.Timestamp 
//display(bars);

### Step 4: Visualize data

#### 4.1 Plot the bars Chart
Plot the bars request result and plot them with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [13]:
var tuples = bars.Select(b => new Tuple<DateTime,double,double,double,double>(b.TimeStamp.ToDateTime(), b.Open, b.High, b.Low, b.Close));

In [14]:
// Plot selected bars
var chart = Chart.Candlestick(tuples);
chart.WithTitle($"{sampling/60}-min Bars for {ticker}");
chart.WithXTitle("Time");
chart.WithYTitle("OHLC");
chart.Width = 1500;
chart.Height = 800;
display(chart);

<!DOCTYPE html>